<a href="https://colab.research.google.com/github/rae-gh/colab-analyses/blob/main/Interpolation_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Interpolation from Unit Cube</h1>
<h4>This interpolation library "interpolates" across infinite space from a unit cell<h4>
(c) Rachel Alcraft 2024

The library was developed to apply electron density across protein structures, and was developed as a result of changes to the EBI deposited electron density - originally the deposited density covered the structure but the depositions have changed to be unit cells (not necessarily even contiguous with the structures).

This library underlies the library `leuci-map` for electron density analysis.  

There are 5 cells to run, you only need to edit the first to change the shape data and plane of interest:
<hr/>
</br> 1. RUN: Import the required librarires
</br> 2. EDIT+RUN: Enter test data, space and interpolation paramaters
</br> 3. Set up the map object
</br> 4. Visualise 2d image slice: values, 1st and 2nd derivative
</br> 5. Visualise projection
<hr/>

In [ ]:
# CELL 1
# PATH TO YOUR SAMPLE DATA (pdbs, results, etc.)
# NOTE if you are using colab there is a deirectory by default called sample_data
# If you are running locally with jupyter notebook choose a directory of your choice

DATADIR = "sample_data/"
IS_GOOGLE_COLAB = True  # Set this to True if you are using Google Colab

In [ ]:
# CELL 2
# Need to import libraries
if IS_GOOGLE_COLAB:  
  try:#runtime gets refreshed so reinstall of non standard libraries may be necessary
    import google.colab    
    !pip install maptial        
  except:
    pass
else:
  print("Not using Google Colab, so make sure you have an environment setup with the correct libraries installed, e.g.\n------------------")
  print("mamba create -n maptial-env python=3.12 ipykernel ipython nbformat")
  print("mamba activate maptial-env")
  print("python -m pip install maptial")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.7/477.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
# CELL 2
###### Edit the data and space ######
vals = [[[1,2,3,4,5],[6,7,8,9,10]],[[11,12,13,14,15],[16,17,18,19,20]]]
interp_method = "bspline" # out of["nearest","linear","mv0","mv1","cubic","bspline"]
width=10
samples=51
# 3 points to define the visual plane
central = (0,0,0)
linear = (1,0,0.33)
planar = (1,0.5,0)

In [ ]:
# CELL 3
### CODE ###
from leuci_xyz import spacetransform as space
from leuci_xyz import gridmaker as grid
from leuci_xyz import vectorthree as v3
import leuci_pol.interpolator as pol

gm = grid.GridMaker()
u_coords = gm.get_unit_grid(width,samples)
# Create the 3 coordinates for orientation
centralxyz = v3.VectorThree(central[0],central[1],central[2])
linearxyz = v3.VectorThree(linear[0],linear[1],linear[2])
planarxyz = v3.VectorThree(planar[0],planar[1],planar[2])
spc = space.SpaceTransform(centralxyz, linearxyz, planarxyz)
xyz_coords = spc.convert_coords(u_coords)
f,m,s = len(vals),len(vals[0]),len(vals[0][0])
intr = pol.create_interpolator(interp_method,vals,FMS=(f,m,s),log_level=1,as_sd=2)
dvals = intr.get_val_slice(xyz_coords,deriv=0,ret_type="2d")
rvals = intr.get_val_slice(xyz_coords,deriv=1,ret_type="2d")
lvals = intr.get_val_slice(xyz_coords,deriv=2,ret_type="2d")

Interpolator: bspline (2, 2, 5)
Mean= 1.8209309360006523
Std= 1.0
Min= 0.17342199390482382
Max= 3.46843987809648


In [ ]:
# CELL 4
# 2d plot (s)
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

# The second derivative has more meaning if we set 0 in the colorscheme
minl,maxl = np.nan, np.nan
for i in range(len(lvals)):
  for j in range(len(lvals[0])):
    minl = min(lvals[i][j],minl)
    maxl = max(lvals[i][j],maxl)
l0 = 0.5
if minl < 0:
  l0 = (0 - minl) / (maxl - minl)
print(minl, maxl, l0)

data_rads = go.Heatmap(z=rvals,colorscale=['Black','Snow'],showscale=False)
data_laps = go.Heatmap(z=lvals,showscale=False, colorscale=[(0, "rgb(100,0,0)"),(l0, "snow"), (1, "navy")],)
data_vals = go.Heatmap(z=dvals,showscale=True, colorscale=[(0, "grey"), (0.1, "snow"), (0.5, "cornflowerblue"),(0.9, "crimson"),(1.0, "rgb(100,0,0)")],)

fig = make_subplots(rows=1, cols=3,horizontal_spacing=0.05,vertical_spacing=0.05,column_widths=[0.33,0.33,0.33])

fig.add_trace(data_vals,row=1,col=1)
fig.add_trace(data_rads,row=1,col=2)
fig.add_trace(data_laps,row=1,col=3)
fig.update_xaxes(showticklabels=False) # hide all the xticks
fig.update_yaxes(showticklabels=False) # hide all the xticks
fig.update_yaxes(scaleanchor="x",scaleratio=1)
fig.update_xaxes(scaleanchor="y",scaleratio=1)



-16.094992982829126 17.49871749168186 0.47910733156556495


In [ ]:
# CELL 5
# 3d plot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

projXY = intr.get_projection("xy", 0,5,0,5)
projYZ = intr.get_projection("yz", 0,5,0,5)
projZX = intr.get_projection("zx", 0,5,0,5)

data_valsXY = go.Heatmap(z=projXY,showscale=True, colorscale=[(0, "grey"), (0.1, "snow"), (0.5, "cornflowerblue"),(0.9, "crimson"),(1.0, "rgb(100,0,0)")],)
data_valsYZ = go.Heatmap(z=projYZ,showscale=False, colorscale=[(0, "grey"), (0.1, "snow"), (0.5, "cornflowerblue"),(0.9, "crimson"),(1.0, "rgb(100,0,0)")],)
data_valsZX = go.Heatmap(z=projZX,showscale=False, colorscale=[(0, "grey"), (0.1, "snow"), (0.5, "cornflowerblue"),(0.9, "crimson"),(1.0, "rgb(100,0,0)")],)

fig = make_subplots(rows=1, cols=3,horizontal_spacing=0.05,vertical_spacing=0.05,column_widths=[0.33,0.33,0.33])

fig.add_trace(data_valsXY,row=1,col=1)
fig.add_trace(data_valsYZ,row=1,col=2)
fig.add_trace(data_valsZX,row=1,col=3)
fig.update_xaxes(showticklabels=False) # hide all the xticks
fig.update_yaxes(showticklabels=False) # hide all the xticks
fig.update_yaxes(scaleanchor="x",scaleratio=1)
fig.update_xaxes(scaleanchor="y",scaleratio=1)